<a href="https://colab.research.google.com/github/Urooj159/Clustering-and-Dimensionality-Reduction-in-Machine-Learning/blob/main/CNNs%26Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 📍 STEP 1: Install Streamlit & Ngrok in Google Colab
!pip install streamlit pyngrok --quiet

# Import Ngrok to create a public link later
from pyngrok import ngrok

# Kill any existing ngrok processes (avoids "address already in use" errors)
ngrok.kill()

# 📍 STEP 2: Training Code (Your Existing Code)
# ======================
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import os

# Example Dataset: Flowers (replace with your dataset path)
# Download small flowers dataset for testing
!wget -q https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
!tar -xzf flower_photos.tgz

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = "/content/flower_photos"

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_data = train_gen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = train_gen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

num_classes = train_data.num_classes
class_names = list(train_data.class_indices.keys())

def build_model(base_model):
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=predictions)

base_model_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = build_model(base_model_mobilenet)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint("model.h5", save_best_only=True)
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=3,  # small for quick test
    callbacks=callbacks
)

# 📍 STEP 3: Create Streamlit UI

%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

# Load model
model = load_model("model.h5")

# Class names
class_names = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

# Preprocessing function
def preprocess_image(img):
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

st.title("🌸 Flower Image Classifier")
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])
if uploaded_file:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)
    processed = preprocess_image(image)
    prediction = model.predict(processed)[0]
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction)
    st.success(f"Prediction: **{predicted_class}** ({confidence:.2f})")

    st.subheader("Top 3 Predictions:")
    top_3_indices = prediction.argsort()[-3:][::-1]
    for i in top_3_indices:
        st.write(f"{class_names[i]}: {prediction[i]:.2f}")


# 📍 STEP 4: Run Streamlit with Ngrok

from pyngrok import ngrok

# Kill previous tunnels
ngrok.kill()

# Start new tunnel
public_url = ngrok.connect(8501)
print(f"Streamlit App URL: {public_url}")

!streamlit run app.py &>/dev/null&


Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.
Epoch 1/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6591 - loss: 0.9273

92/92 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.6601 - loss: 0.9246 - val_accuracy: 0.8345 - val_loss: 0.4744
Epoch 2/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8518 - loss: 0.3965

92/92 ━━━━━━━━━━━━━━━━━━━━ 185s 2s/step - accuracy: 0.8518 - loss: 0.3965 - val_accuracy: 0.8482 - val_loss: 0.4366
Epoch 3/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8743 - loss: 0.3550

92/92 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.8743 - loss: 0.3550 - val_accuracy: 0.8536 - val_loss: 0.3989


UsageError: Line magic function `%%writefile` not found.
